In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기

In [2]:
train = pd.read_csv('C:/dacon/train.csv')
test = pd.read_csv('C:/dacon/test.csv')
submission = pd.read_csv('C:/dacon/sample_submission.csv')
car_2020 = pd.read_csv('C:/dacon/car_2020.csv', encoding = 'cp949')

In [3]:
car_2020.head(5)

,시도(1),시군구(1),레벨01(1),2020. 01,2020. 01.1,2020. 01.2,2020. 01.3,2020. 02,2020. 02.1,2020. 02.2,...,2020. 10.2,2020. 10.3,2020. 11,2020. 11.1,2020. 11.2,2020. 11.3,2020. 12,2020. 12.1,2020. 12.2,2020. 12.3
0,시도(1),시군구(1),레벨01(1),관용,자가용,영업용,계,관용,자가용,영업용,...,영업용,계,관용,자가용,영업용,계,관용,자가용,영업용,계
1,서울,계,승용,4576,2542280,127057,2673913,4586,2540857,126079,...,125280,2705842,4641,2580570,125216,2710427,4607,2582001,125376,2711984
2,서울,계,승합,3475,92965,17565,114005,3485,92398,17481,...,15432,107701,3740,87964,15412,107116,3805,87315,15384,106504
3,서울,계,화물,3968,269582,57819,331369,3987,268992,57773,...,57958,331371,3969,268864,58018,330851,3937,267951,58055,329943
4,서울,계,특수,354,2961,4964,8279,351,3015,4980,...,4942,8818,372,3564,4948,8884,374,3625,4931,8930


In [4]:
# 0번째 인덱스 삭제
car_2020 = car_2020.drop(index=0)

In [5]:
#시군구 컬럼 삭제
car_2020 = car_2020.drop(columns='시군구(1)')
car_2020.head(5)

,시도(1),레벨01(1),2020. 01,2020. 01.1,2020. 01.2,2020. 01.3,2020. 02,2020. 02.1,2020. 02.2,2020. 02.3,...,2020. 10.2,2020. 10.3,2020. 11,2020. 11.1,2020. 11.2,2020. 11.3,2020. 12,2020. 12.1,2020. 12.2,2020. 12.3
1,서울,승용,4576,2542280,127057,2673913,4586,2540857,126079,2671522,...,125280,2705842,4641,2580570,125216,2710427,4607,2582001,125376,2711984
2,서울,승합,3475,92965,17565,114005,3485,92398,17481,113364,...,15432,107701,3740,87964,15412,107116,3805,87315,15384,106504
3,서울,화물,3968,269582,57819,331369,3987,268992,57773,330752,...,57958,331371,3969,268864,58018,330851,3937,267951,58055,329943
4,서울,특수,354,2961,4964,8279,351,3015,4980,8346,...,4942,8818,372,3564,4948,8884,374,3625,4931,8930
5,서울,총계,12373,2907788,207405,3127566,12409,2905262,206313,3123984,...,203612,3153732,12722,2940962,203594,3157278,12723,2940892,203746,3157361


In [6]:
#차량 종류 컬럼 삭제
car_2020 = car_2020.drop(columns = "레벨01(1)")
car_2020.head(5)

,시도(1),2020. 01,2020. 01.1,2020. 01.2,2020. 01.3,2020. 02,2020. 02.1,2020. 02.2,2020. 02.3,2020. 03,...,2020. 10.2,2020. 10.3,2020. 11,2020. 11.1,2020. 11.2,2020. 11.3,2020. 12,2020. 12.1,2020. 12.2,2020. 12.3
1,서울,4576,2542280,127057,2673913,4586,2540857,126079,2671522,4507,...,125280,2705842,4641,2580570,125216,2710427,4607,2582001,125376,2711984
2,서울,3475,92965,17565,114005,3485,92398,17481,113364,3551,...,15432,107701,3740,87964,15412,107116,3805,87315,15384,106504
3,서울,3968,269582,57819,331369,3987,268992,57773,330752,4022,...,57958,331371,3969,268864,58018,330851,3937,267951,58055,329943
4,서울,354,2961,4964,8279,351,3015,4980,8346,352,...,4942,8818,372,3564,4948,8884,374,3625,4931,8930
5,서울,12373,2907788,207405,3127566,12409,2905262,206313,3123984,12432,...,203612,3153732,12722,2940962,203594,3157278,12723,2940892,203746,3157361


In [7]:
#지역으로 컬럼명 변경
car_2020.rename(columns={'시도(1)':'지역'},inplace=True)
car_2020.head(5)

,지역,2020. 01,2020. 01.1,2020. 01.2,2020. 01.3,2020. 02,2020. 02.1,2020. 02.2,2020. 02.3,2020. 03,...,2020. 10.2,2020. 10.3,2020. 11,2020. 11.1,2020. 11.2,2020. 11.3,2020. 12,2020. 12.1,2020. 12.2,2020. 12.3
1,서울,4576,2542280,127057,2673913,4586,2540857,126079,2671522,4507,...,125280,2705842,4641,2580570,125216,2710427,4607,2582001,125376,2711984
2,서울,3475,92965,17565,114005,3485,92398,17481,113364,3551,...,15432,107701,3740,87964,15412,107116,3805,87315,15384,106504
3,서울,3968,269582,57819,331369,3987,268992,57773,330752,4022,...,57958,331371,3969,268864,58018,330851,3937,267951,58055,329943
4,서울,354,2961,4964,8279,351,3015,4980,8346,352,...,4942,8818,372,3564,4948,8884,374,3625,4931,8930
5,서울,12373,2907788,207405,3127566,12409,2905262,206313,3123984,12432,...,203612,3153732,12722,2940962,203594,3157278,12723,2940892,203746,3157361


In [10]:
#관용 자동차 컬럼 삭제
car_2020=car_2020.drop(columns =['2020. 01', '2020. 02','2020. 03','2020. 04','2020. 05','2020. 06', '2020. 07', '2020. 08', '2020. 09', '2020. 10', '2020. 11', '2020. 12'])


In [12]:
car_2020.rename(columns={'2020. 01.1':'자가용1월','2020. 01.2':'영업용1월'},inplace=True)
car_2020.rename(columns={'2020. 02.1':'자가용2월','2020. 02.2':'영업용2월'},inplace=True)
car_2020.rename(columns={'2020. 03.1':'자가용3월','2020. 03.2':'영업용3월'},inplace=True)
car_2020.rename(columns={'2020. 04.1':'자가용4월','2020. 04.2':'영업용4월'},inplace=True)
car_2020.rename(columns={'2020. 05.1':'자가용5월','2020. 05.2':'영업용5월'},inplace=True)
car_2020.rename(columns={'2020. 06.1':'자가용6월','2020. 06.2':'영업용6월'},inplace=True)
car_2020.rename(columns={'2020. 07.1':'자가용7월','2020. 07.2':'영업용7월'},inplace=True)
car_2020.rename(columns={'2020. 08.1':'자가용8월','2020. 08.2':'영업용8월'},inplace=True)
car_2020.rename(columns={'2020. 09.1':'자가용9월','2020. 09.2':'영업용9월'},inplace=True)
car_2020.rename(columns={'2020. 10.1':'자가용10월','2020. 10.2':'영업용10월'},inplace=True)
car_2020.rename(columns={'2020. 11.1':'자가용11월','2020. 11.2':'영업용11월'},inplace=True)
car_2020.rename(columns={'2020. 12.1':'자가용12월','2020. 12.2':'영업용12월'},inplace=True)

In [14]:
car_2020.rename(columns={'2020. 01.3':'1월총합'},inplace=True)
car_2020.rename(columns={'2020. 02.3':'2월총합'},inplace=True)
car_2020.rename(columns={'2020. 03.3':'3월총합'},inplace=True)
car_2020.rename(columns={'2020. 04.3':'4월총합'},inplace=True)
car_2020.rename(columns={'2020. 05.3':'5월총합'},inplace=True)
car_2020.rename(columns={'2020. 06.3':'6월총합'},inplace=True)
car_2020.rename(columns={'2020. 07.3':'7월총합'},inplace=True)
car_2020.rename(columns={'2020. 08.3':'8월총합'},inplace=True)
car_2020.rename(columns={'2020. 09.3':'9월총합'},inplace=True)
car_2020.rename(columns={'2020. 10.3':'10월총합'},inplace=True)
car_2020.rename(columns={'2020. 11.3':'11월총합'},inplace=True)
car_2020.rename(columns={'2020. 12.3':'12월총합'},inplace=True)

car_2020.head(5)

,지역,자가용1월,영업용1월,1월총합,자가용2월,영업용2월,2월총합,자가용3월,영업용3월,3월총합,...,9월총합,자가용10월,영업용10월,10월총합,자가용11월,영업용11월,11월총합,자가용12월,영업용12월,12월총합
1,서울,2542280,127057,2673913,2540857,126079,2671522,2543567,125118,2673192,...,2703429,2576019,125280,2705842,2580570,125216,2710427,2582001,125376,2711984
2,서울,92965,17565,114005,92398,17481,113364,91798,17308,112657,...,108205,88509,15432,107701,87964,15412,107116,87315,15384,106504
3,서울,269582,57819,331369,268992,57773,330752,269312,57762,331096,...,331429,269398,57958,331371,268864,58018,330851,267951,58055,329943
4,서울,2961,4964,8279,3015,4980,8346,3052,4984,8388,...,8779,3502,4942,8818,3564,4948,8884,3625,4931,8930
5,서울,2907788,207405,3127566,2905262,206313,3123984,2907729,205172,3125333,...,3151842,2937428,203612,3153732,2940962,203594,3157278,2940892,203746,3157361


In [22]:
car_2020.columns

Index(['지역', '자가용1월', '영업용1월', '1월총합', '자가용2월', '영업용2월', '2월총합', '자가용3월',
       '영업용3월', '3월총합', '자가용4월', '영업용4월', '4월총합', '자가용5월', '영업용5월', '5월총합',
       '자가용6월', '영업용6월', '6월총합', '자가용7월', '영업용7월', '7월총합', '자가용8월', '영업용8월',
       '8월총합', '자가용9월', '영업용9월', '9월총합', '자가용10월', '영업용10월', '10월총합', '자가용11월',
       '영업용11월', '11월총합', '자가용12월', '영업용12월', '12월총합'],
      dtype='object')

In [59]:
len(train[(train['전용면적']>60) & (train['임대건물구분'] == '아파트') & (train['신분']=='D')]['전용면적'])
#85안넘어감  
'''60넘어가는 신분 D 아파트 : 7개'''

7

In [60]:
len(train[(train['전용면적']>60) & (train['임대건물구분'] == '아파트') & (train['신분']=='A')]['전용면적'])
#85안넘어감  
'''60 넘어가는 신분A 아파트 :  131개'''

131

In [64]:
# train[(train['전용면적']>60) & (train['임대건물구분'] == '상가')]['신분'].unique() 
'''상가 모두 자격 D'''

'''60 안넘어가는 아파트: 2252개
   60 안넘어가는
'''

train[(train['전용면적']<60) & (train['임대건물구분'] == '아파트')]['신분'].unique()

array(['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O'], dtype=object)

# 컬렴명 바꿔주기

In [23]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]


# 데이터 전처리

In [24]:
# 지하철이 없으면 0, 있으면 1 로 데이터 레이블 작업 진행

train['지하철'] = train['지하철'].fillna(0)
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)
train['지하철'].value_counts()

'''
idx = train[train['지하철']==0].index
train.loc[idx,'지하철'] = 0
idx = train[train['지하철']!=0].index
train.loc[idx,['지하철']] = 1
'''

"\nidx = train[train['지하철']==0].index\ntrain.loc[idx,'지하철'] = 0\nidx = train[train['지하철']!=0].index\ntrain.loc[idx,['지하철']] = 1\n"

In [25]:
train['지하철'].value_counts()

0    2513
1     439
Name: 지하철, dtype: int64

In [26]:
# 원핫 인코딩
#train = pd.get_dummies(data = train, columns = ['지하철'])
train['공급유형'].unique()
train['신분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O'], dtype=object)

# 결측치 처리

In [27]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

In [28]:
train['버스'] = train['버스'].fillna(train['버스'].mean())

In [29]:
test.loc[test['임대건물구분'] == '상가'].isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,0
임대보증금,177


In [30]:
train['임대건물구분'].unique()

array(['아파트', '상가'], dtype=object)

In [31]:
train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [32]:
train['임대료']

0       103680.0
1       103680.0
2       184330.0
3       184330.0
4       184330.0
          ...   
2947    116090.0
2948    142310.0
2949    142310.0
2950    142310.0
2951    151030.0
Name: 임대료, Length: 2952, dtype: float64

In [11]:
test.loc[test['단지코드']==1350]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수


In [20]:
train[(train['신분']=='D') & (train['임대건물구분'] == '상가')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
88,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,0.0,0.0,0,4.0,117.0,75.0
89,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,0.0,0.0,0,4.0,117.0,75.0
90,C1925,601,상가,강원도,임대상가,32.10,1,9.0,D,0.0,0.0,0,4.0,117.0,75.0
91,C1925,601,상가,강원도,임대상가,72.16,1,9.0,D,0.0,0.0,0,4.0,117.0,75.0
101,C1874,619,상가,충청남도,임대상가,12.62,1,2.0,D,0.0,0.0,0,2.0,97.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,C2571,481,상가,경상남도,임대상가,31.84,1,5.0,D,0.0,0.0,0,1.0,65.0,62.0
835,C2571,481,상가,경상남도,임대상가,31.84,1,5.0,D,0.0,0.0,0,1.0,65.0,62.0
836,C2571,481,상가,경상남도,임대상가,31.84,1,5.0,D,0.0,0.0,0,1.0,65.0,62.0
837,C2571,481,상가,경상남도,임대상가,31.84,1,5.0,D,0.0,0.0,0,1.0,65.0,62.0


In [17]:
train[(train['신분']=='D') & (train['임대건물구분'] == '아파트')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,0.0,0.0,0,6.0,1636.0,2315.0


# 지역명 숫자로 매핑

In [6]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

# 전용면적을 5의 배수로 변경

In [7]:
train['전용면적'] = train['전용면적']//5*5

# 전용면적 상/하한 적용

In [8]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2952 non-null   object 
 1   총세대수      2952 non-null   int64  
 2   임대건물구분    2952 non-null   object 
 3   지역        2952 non-null   int64  
 4   공급유형      2952 non-null   object 
 5   전용면적      2952 non-null   float64
 6   전용면적별세대수  2952 non-null   int64  
 7   공가수       2952 non-null   float64
 8   신분        2952 non-null   object 
 9   임대보증금     2952 non-null   float64
 10  임대료       2952 non-null   float64
 11  지하철       2952 non-null   float64
 12  버스        2952 non-null   float64
 13  단지내주차면수   2952 non-null   float64
 14  등록차량수     2952 non-null   float64
dtypes: float64(8), int64(3), object(4)
memory usage: 346.1+ KB


In [10]:
from sklearn.model_selection import train_test_split

x = train.drop(['등록차량수','단지코드','임대건물구분','공급유형','신분'], axis=1) 
y = train['등록차량수']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

display(x_train.shape, x_test.shape)

(2214, 10)

(738, 10)

In [11]:
forest = RandomForestRegressor(n_jobs=-1, random_state=42)
forest.fit(x_train, y_train).score(x_test, y_test)

0.9891439162587817

# test 불러오기

In [22]:
test[test['단지코드'] == 'C1327']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
891,C1327,98,아파트,대전광역시,행복주택,16.83,30,13.0,J,18800000,94000,NaN,17.0,98.0
892,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000,142000,NaN,17.0,98.0
893,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000,142000,NaN,17.0,98.0
894,C1327,98,아파트,대전광역시,행복주택,36.92,48,13.0,J,36800000,184000,NaN,17.0,98.0


In [12]:
test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [13]:
test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
신분            2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

In [14]:
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)

In [15]:
test['지하철'] = test['지하철'].fillna(test['지하철'].mean())

In [16]:
test.loc[test.단지코드.isin(['C2411']) & test.신분.isnull(), '신분'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.신분.isnull(), '신분'] = 'C'

In [17]:
test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64

In [18]:
local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

In [19]:
x_train

,총세대수,지역,전용면적,전용면적별세대수,공가수,임대보증금,임대료,지하철,버스,단지내주차면수
303,809,5,45.0,149,1.0,13101000.0,221030.0,0.000000,6.0,264.0
132,560,1,30.0,192,0.0,3631000.0,80800.0,0.000000,5.0,154.0
1851,1668,2,35.0,11,17.0,24775000.0,172190.0,1.000000,10.0,1756.0
1791,1035,3,50.0,108,21.0,23367000.0,237570.0,0.000000,16.0,911.0
526,2428,8,15.0,1,1.0,0.0,0.0,0.000000,1.0,756.0
...,...,...,...,...,...,...,...,...,...,...
763,1144,7,55.0,240,16.0,18770000.0,246770.0,0.176578,8.0,950.0
835,481,1,30.0,1,5.0,0.0,0.0,0.000000,1.0,65.0
1653,511,9,50.0,29,3.0,18968000.0,207380.0,0.000000,4.0,413.0
2607,902,6,40.0,48,32.0,41120000.0,233010.0,0.000000,2.0,635.0


In [20]:
columns = ['총세대수', '지역',' 전용면적','전용면적별세대수','공가수','임대보증금','임대료','지하철','버스','단지내주차면수']
new_test = pd.DataFrame()
area_columns = []

In [21]:
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

0it [00:00, ?it/s]


KeyError: ' 전용면적'

In [34]:
new_test.head

<bound method NDFrame.head of       단지코드    총세대수   공가수    지역  단지내주차면수  지하철    버스
0    C1072   754.0  14.0   0.0    683.0  0.0   2.0
1    C1128  1354.0   9.0   0.0   1216.0  0.0   3.0
2    C1456   619.0  18.0   1.0    547.0  0.0  16.0
3    C1840   593.0   7.0   2.0    543.0  0.0   3.0
4    C1332  1297.0  11.0   0.0   1112.0  0.0   2.0
..     ...     ...   ...   ...      ...  ...   ...
145  C2456   349.0  17.0   6.0    270.0  0.0   4.0
146  C1266   596.0  35.0  10.0    593.0  0.0   1.0
147  C2152   120.0   9.0   7.0     40.0  0.0   1.0
148  C1267   675.0  38.0   3.0    467.0  0.0   1.0
149  C2189   382.0  45.0   2.0    300.0  0.0   2.0

[150 rows x 7 columns]>

In [24]:
pred = forest.predict(x_test)
#pred

In [25]:
submission['num'] = pred

ValueError: Length of values (738) does not match length of index (150)

In [ ]:
submission.to_csv('baseline.csv', index=False)